In [1]:
mountedDB = {}

In [2]:
!pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
!pip install easydict

Writing to /home/mw/.config/pip/pip.conf
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
You are using pip version 18.1, however version 24.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import time
import torch.backends.cudnn as cudnn
from torch import nn
from easydict import EasyDict as edict
from models import Generator, Discriminator, TruncatedVGG19
from datasets import SRDataset
from utils import *
from solver import train

In [4]:
# config
config = edict()
config.csv_folder = '/home/mw/project/SRDataset'
config.HR_data_folder = '/home/mw/input/dataset76853/DIV2K_train_HR/DIV2K_train_HR'
config.LR_data_folder = '/home/mw/input/dataset76853/DIV2K_train_LR_bicubic_X4/DIV2K_train_LR_bicubic/X4'
config.crop_size = 96
config.scaling_factor = 4

# Generator parameters
config.G = edict()
config.G.large_kernel_size = 9
config.G.small_kernel_size = 3
config.G.n_channels = 64
config.G.n_blocks = 16
srresnet_checkpoint = "/home/mw/project/checkpoint_srresnet.pth.tar"

# Discriminator parameters
config.D = edict()
config.D.kernel_size = 3
config.D.n_channels = 64
config.D.n_blocks = 8
config.D.fc_size = 1024

# Learning parameters
config.checkpoint = None # path to model (SRGAN) checkpoint, None if none
config.batch_size = 16
config.start_epoch = 0
config.epochs = 20
config.workers = 4
config.vgg19_i = 5  # the index i in the definition for VGG loss; see paper
config.vgg19_j = 4  # the index j in the definition for VGG loss; see paper
config.beta = 1e-3  # the coefficient to weight the adversarial loss in the perceptual loss
config.print_freq = 50
config.lr = 1e-4

# Default device
config.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cudnn.benchmark = True

In [5]:
if config.checkpoint is None:
    # Generator
    generator = Generator(config)
    generator.initialize_with_srresnet(srresnet_checkpoint=srresnet_checkpoint)

    # Initialize generator's optimizer
    optimizer_g = torch.optim.Adam(params=filter(lambda p: p.requires_grad, generator.parameters()),
                                   lr=config.lr)

    # Discriminator
    discriminator = Discriminator(config)
    optimizer_d = torch.optim.Adam(params=filter(lambda p: p.requires_grad, discriminator.parameters()),
                                   lr=config.lr)

else:
    checkpoint = torch.load(config.checkpoint)
    config.start_epoch = checkpoint['epoch'] + 1
    generator = checkpoint['generator']
    discriminator = checkpoint['discriminator']
    optimizer_g = checkpoint['optimizer_g']
    optimizer_d = checkpoint['optimizer_d']
    print("\nLoaded checkpoint from epoch %d.\n" % (checkpoint['epoch'] + 1))


Loaded weights from pre-trained SRResNet.



In [6]:
# Truncated VGG19 network to be used in the loss calculation
weights_path = '/home/mw/work/work_dir/vgg19-dcbb9e9d.pth'
truncated_vgg19 = TruncatedVGG19(i=config.vgg19_i, j=config.vgg19_j, weights_path = weights_path)
truncated_vgg19.eval()

TruncatedVGG19(
  (truncated_vgg19): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), s

In [7]:
# Loss functions
content_loss_criterion = nn.MSELoss()
adversarial_loss_criterion = nn.BCEWithLogitsLoss()

In [8]:
# Move to default device
generator = generator.to(config.device)
discriminator = discriminator.to(config.device)
truncated_vgg19 = truncated_vgg19.to(config.device)
content_loss_criterion = content_loss_criterion.to(config.device)
adversarial_loss_criterion = adversarial_loss_criterion.to(config.device)

In [9]:
# Custom dataloaders
train_dataset = SRDataset(split='train', config=config)
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=config.batch_size,
                                           shuffle=True, 
                                           num_workers=config.workers,
                                           pin_memory=True)

In [10]:
# Epochs
for epoch in range(config.start_epoch, config.epochs):
    # At the halfway point, reduce learning rate to a tenth
    if epoch == int(config.epochs / 2 + 1):
        adjust_learning_rate(optimizer_g, 0.1)
        adjust_learning_rate(optimizer_d, 0.1)
    # One epoch's training
    train(train_loader=train_loader,
          generator=generator,
          discriminator=discriminator,
          truncated_vgg19=truncated_vgg19,
          content_loss_criterion=content_loss_criterion,
          adversarial_loss_criterion=adversarial_loss_criterion,
          optimizer_g=optimizer_g,
          optimizer_d=optimizer_d,
          epoch=epoch,
          device=config.device,
          beta=config.beta,
          print_freq=config.print_freq)
    # Save checkpoint
    torch.save({'epoch': epoch,
                'generator': generator,
                'discriminator': discriminator,
                'optimizer_g': optimizer_g,
                'optimizer_d': optimizer_d},
                'checkpoint_srgan.pth.tar')

Epoch: [0][0/5000]----Batch Time 3.804 (3.804)----Data Time 1.875 (1.875)----Cont. Loss 0.0012 (0.0012)----Adv. Loss 0.6325 (0.6325)----Disc. Loss 1.3881 (1.3881)
Epoch: [0][50/5000]----Batch Time 0.497 (0.446)----Data Time 0.305 (0.219)----Cont. Loss 0.0007 (0.0005)----Adv. Loss 2.2823 (1.1392)----Disc. Loss 0.9539 (1.2278)
Epoch: [0][100/5000]----Batch Time 0.606 (0.432)----Data Time 0.413 (0.221)----Cont. Loss 0.0016 (0.0008)----Adv. Loss 6.3095 (1.8791)----Disc. Loss 0.7072 (1.0353)
Epoch: [0][150/5000]----Batch Time 0.324 (0.427)----Data Time 0.130 (0.221)----Cont. Loss 0.0003 (0.0007)----Adv. Loss 0.9776 (1.9759)----Disc. Loss 1.2526 (1.0933)
Epoch: [0][200/5000]----Batch Time 0.197 (0.429)----Data Time 0.000 (0.225)----Cont. Loss 0.0004 (0.0007)----Adv. Loss 2.3515 (1.8117)----Disc. Loss 1.8138 (1.1051)
Epoch: [0][250/5000]----Batch Time 0.196 (0.426)----Data Time 0.000 (0.224)----Cont. Loss 0.0007 (0.0006)----Adv. Loss 0.8583 (1.6739)----Disc. Loss 1.0943 (1.1234)
Epoch: [0][30